In [1]:
from diskos.reader import build_reader
import diskos.spec as spec
from diskos.sender_specific.merge import AptAnalId

In [2]:
reader = build_reader(r'c:\data\diskos\6406_12-3_S__ROCK_AND_CORE__GCH_RAW_1.ASC', quiet_mode=True)
spec.SUPPRESS_WARNINGS = True  # some rows have missing cells - we know
parsed_file =reader.parse_file()
block_26 = next((b for b in parsed_file.blocks if b.block_def.block_no == '26'))
orig = block_26.as_dataframe
orig

,AnalID,d13C
0,134904WCSATA,-28.9
1,134904WCAROA,-27.9
2,144817WCSATA,-28.9
3,144817WCAROA,-28.0
4,10042WCWHOA,-28.8
5,10042WCWHOB,-28.7
6,10042WCWHOC,-28.7
7,10042WCWHOD,-28.7


In [3]:
cols_to_expand = [c for c in orig.columns if c != 'AnalID']
cols_to_expand

['d13C']

In [4]:
df = orig.copy()
df['AnalCode'] = df['AnalID'].apply(lambda id: AptAnalId(id).anal_code)
unique_anal_codes = df['AnalCode'].unique()

In [5]:
for code in unique_anal_codes:
    for col in cols_to_expand:
        df[f"{col}_{code}"] = df.apply(lambda row: row[col] if row['AnalCode'] == code else '', axis=1)

In [6]:
df

,AnalID,d13C,AnalCode,d13C_CSAT,d13C_CARO,d13C_CWHO
0,134904WCSATA,-28.9,CSAT,-28.9,,
1,134904WCAROA,-27.9,CARO,,-27.9,
2,144817WCSATA,-28.9,CSAT,-28.9,,
3,144817WCAROA,-28.0,CARO,,-28.0,
4,10042WCWHOA,-28.8,CWHO,,,-28.8
5,10042WCWHOB,-28.7,CWHO,,,-28.7
6,10042WCWHOC,-28.7,CWHO,,,-28.7
7,10042WCWHOD,-28.7,CWHO,,,-28.7


In [7]:
#merge rows
# get igi id 
df['IgiID'] = df['AnalID'].apply(lambda anal_id: AptAnalId(anal_id).igi_id)
#rmv anal_id and uneeded cols used for expansion
df.drop(columns=['AnalID', 'd13C', 'AnalCode'], inplace=True)
df

,d13C_CSAT,d13C_CARO,d13C_CWHO,IgiID
0,-28.9,,,134904WA
1,,-27.9,,134904WA
2,-28.9,,,144817WA
3,,-28.0,,144817WA
4,,,-28.8,10042WA
5,,,-28.7,10042WB
6,,,-28.7,10042WC
7,,,-28.7,10042WD


In [8]:
#squash rows (csan't find a safer way to do this - i.e. only comb if one value)
merged_df = df.groupby('IgiID').aggregate(''.join)
merged_df

,d13C_CSAT,d13C_CARO,d13C_CWHO
IgiID,,,
10042WA,,,-28.8
10042WB,,,-28.7
10042WC,,,-28.7
10042WD,,,-28.7
134904WA,-28.9,-27.9,
144817WA,-28.9,-28.0,


In [12]:
merged_df.index.name

'IgiID'